In [1]:
path_main = r"E:\ML algoritme tl anfaldsdetektion vha HRV\LabView-Results_Excluded_seizures_removed"

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [7]:
import json

# helper to find n_samples anywhere in a nested json
def find_n_samples(x):
    if isinstance(x, dict):
        if "n_samples" in x:
            return x["n_samples"]
        for v in x.values():
            res = find_n_samples(v)
            if res is not None:
                return res
    elif isinstance(x, list):
        for item in x:
            res = find_n_samples(item)
            if res is not None:
                return res
    return None

# helper to format seconds into days/hours/minutes/seconds
def format_duration(sec):
    sec = float(sec)
    days = int(sec // 86400)
    sec -= days * 86400
    hours = int(sec // 3600)
    sec -= hours * 3600
    minutes = int(sec // 60)
    sec -= minutes * 60
    return f"{days}d {hours}h {minutes}m {sec:.2f}s"

# collect per-patient and per-group samples
per_patient = {"responders": {}, "nonresponders": {}}

for group in list(per_patient.keys()):
    group_dir = os.path.join(path_main, group)
    if not os.path.isdir(group_dir):
        continue
    for patient_name in os.listdir(group_dir):
        patient_dir = os.path.join(group_dir, patient_name)
        if not os.path.isdir(patient_dir):
            continue
        patient_samples = []
        for root, _, files in os.walk(patient_dir):
            for fname in files:
                if "meta" in fname.lower():
                    fp = os.path.join(root, fname)
                    try:
                        with open(fp, "r", encoding="utf-8") as fh:
                            obj = json.load(fh)
                    except Exception:
                        continue
                    n = find_n_samples(obj)
                    if n is not None:
                        try:
                            patient_samples.append(int(n))
                        except Exception:
                            pass
        if patient_samples:
            per_patient[group][patient_name] = np.array(patient_samples, dtype=int)

# print per-patient and per-group summaries with human-readable durations
for group, patients in per_patient.items():
    print(f"Group: {group}")
    group_samples_list = []
    for pname, samples in sorted(patients.items()):
        total_samples = samples.sum()
        total_seconds = total_samples / sr
        rec_count = samples.size
        mean_seconds = samples.mean() / sr if rec_count else 0
        group_samples_list.append(samples)
        print(
            f"  {pname}: recordings={rec_count}, total_samples={total_samples}, "
            f"total_seconds={total_seconds:.2f} ({format_duration(total_seconds)}), "
            f"mean_seconds_per_recording={mean_seconds:.2f} ({format_duration(mean_seconds)})"
        )
    # group-level aggregate
    if group_samples_list:
        # ensure we sum in 64-bit to avoid overflow on large totals
        all_samples = np.concatenate(group_samples_list).astype(np.int64)
        total_samples = int(all_samples.sum())
        grp_total_sec = total_samples / sr
        grp_mean_sec = all_samples.mean() / sr
        print(
            f"  -> Group summary: recordings={all_samples.size}, total_samples={total_samples}, "
            f"total_seconds={grp_total_sec:.2f} ({format_duration(grp_total_sec)}), "
            f"mean_seconds_per_recording={grp_mean_sec:.2f} ({format_duration(grp_mean_sec)})"
        )
    else:
        print("  -> No recordings found for this group.")
    print()

Group: responders
  Patient 10: recordings=1, total_samples=132710144, total_seconds=259199.50 (2d 23h 59m 59.50s), mean_seconds_per_recording=259199.50 (2d 23h 59m 59.50s)
  Patient 14: recordings=2, total_samples=175093504, total_seconds=341979.50 (3d 22h 59m 39.50s), mean_seconds_per_recording=170989.75 (1d 23h 29m 49.75s)
  Patient 15: recordings=2, total_samples=224449920, total_seconds=438378.75 (5d 1h 46m 18.75s), mean_seconds_per_recording=219189.38 (2d 12h 53m 9.38s)
  Patient 16: recordings=1, total_samples=78981760, total_seconds=154261.25 (1d 18h 51m 1.25s), mean_seconds_per_recording=154261.25 (1d 18h 51m 1.25s)
  Patient 21: recordings=1, total_samples=88078592, total_seconds=172028.50 (1d 23h 47m 8.50s), mean_seconds_per_recording=172028.50 (1d 23h 47m 8.50s)
  Patient 23: recordings=1, total_samples=91258880, total_seconds=178240.00 (2d 1h 30m 40.00s), mean_seconds_per_recording=178240.00 (2d 1h 30m 40.00s)
  Patient 27: recordings=1, total_samples=123695488, total_seco

In [ ]:
# Mangler 6, 10, 12, 33, 38
# Rettet tidspunkter, forkerte tegn, slåfejl i mappenavne og filnavne, slåfejl i annoteringsfiler.